## Demo for SDK

This example shows how to use SDK to deploy a task. The demo notebook includes the following steps:
- [initialization](#initialization)
- [show available hardware info list](#show-available-hardware-information)
- [choose hardware config](#choose-hardware-config)
- [get `job_source_uri`](#get-job_source_uri)
- [estimate payment amount](#estimate-payment-amount)
- [create task to get `task_uuid`](#create-task)
- [submit payment](#submit-payment)
- [validate payment](#validate-payment-to-deploy-task)
- [follow up task status](#follow-up-task-status-optional)
- [show result](#show-result)



### Initialization

#### get an `API_KEY`

- For test version, get `API_KEY` in dashboard page: https://orchestrator-test.swanchain.io
- For prod version, get `API_KEY` in dashboard page: https://orchestrator.swanchain.io

If use this repository to test on your local machine, add `sys.path.insert(0, '..')` at the beginning, and run code in the root directory of this repository.

To use this SDK, you need to add environment file `.env` in your local directory, including the following parameters (`PK` is private key):

```
API_KEY=
WALLET=
PK=
```

In [1]:
import sys
sys.path.insert(0, '..') 

import os
import time
import dotenv
import json
dotenv.load_dotenv()

import swan

# dev_url = "https://swanhub-cali.swanchain.io"
# swan.setup_default_session(login_url=dev_url)
# session = swan._get_default_session()

In [2]:
wallet_address = os.getenv('WALLET')
primary_key = os.getenv('PK')
swan_orchestrator = swan.resource(service_name='Orchestrator')

In [ ]:
r = swan_orchestrator.get_premade_image("hello-world")
print(json.dumps(r, indent=2))

In [2]:
swan_orchestrator = swan.resource(service_name='Orchestrator', url_endpoint=dev_url)

NameError: name 'dev_url' is not defined

In [4]:
swan_orchestrator = swan.resource(os.getenv("API"), 'Orchestrator')

In [3]:
r = swan_orchestrator.contract_info
print(json.dumps(r, indent=2))

{
  "client_contract_address": "0x9c5397F804f6663326151c81bBD82bb1451059E8",
  "payment_contract_address": "0xB48c5D1c025655BA79Ac4E10C0F19523dB97c816",
  "rpc_url": "https://rpc-atom-internal.swanchain.io",
  "swan_token_contract_address": "0x91B25A65b295F0405552A4bbB77879ab5e38166c"
}


### Show available hardware information

In [5]:
hardwares_info = swan_orchestrator.get_hardware_config()
hardwares_info

[{'id': 0,
  'name': 'C1ae.small',
  'description': 'CPU only · 2 vCPU · 2 GiB',
  'type': 'CPU',
  'region': ['Quebec-CA', 'North Carolina-US'],
  'price': '0.0',
  'status': 'available'},
 {'id': 1,
  'name': 'C1ae.medium',
  'description': 'CPU only · 4 vCPU · 4 GiB',
  'type': 'CPU',
  'region': ['Quebec-CA', 'North Carolina-US'],
  'price': '1.0',
  'status': 'available'},
 {'id': 12,
  'name': 'G1ae.small',
  'description': 'Nvidia 3080 · 4 vCPU · 8 GiB',
  'type': 'GPU',
  'region': ['Quebec-CA', 'North Carolina-US'],
  'price': '10.0',
  'status': 'available'},
 {'id': 13,
  'name': 'G1ae.medium',
  'description': 'Nvidia 3080 · 8 vCPU · 16 GiB',
  'type': 'GPU',
  'region': ['Quebec-CA', 'North Carolina-US'],
  'price': '11.0',
  'status': 'available'},
 {'id': 20,
  'name': 'Hpc1ae.small',
  'description': 'Nvidia 3090 · 4 vCPU · 8 GiB',
  'type': 'GPU',
  'region': ['Quebec-CA'],
  'price': '14.0',
  'status': 'available'},
 {'id': 21,
  'name': 'Hpc1ae.medium',
  'descripti

### Choose hardware config

choose the hardware with its config name or hardware id

in hardware config, `cfg_name` and `hardware_id` will be used in the steps to deploy task (create task and submit payment)

In [3]:
hardware_id = 0 # 'C1ae.small'
region = 'North Carolina-US' #'Quebec-CA'
if swan_orchestrator.set_default_task_config(hardware_id, region):
    print("Successfully set up default task configuration")

Successfully set up default task configuration


### Get `job_source_uri`

We can use the `get_source_uri` interface to get a `job_source_uri` for creating task.

In [4]:

job_source_uri = swan_orchestrator.get_source_uri(
    repo_uri='https://github.com/alphaflows/tetris-docker-image.git',
    wallet_address=wallet_address
    # hardware_id=hardware_id,
)

In [5]:
job_source_uri

'https://plutotest.acl.swanipfs.com/ipfs/QmUiWh2HuJT43RND6TJKmo87poQgDMX9uBu3LDmNnWvuJW'

### Estimate Payment amount

In [7]:
duration_hour = 1 # hour
amount = swan_orchestrator.estimate_payment(
    duration_hour=duration_hour
    # hardware_id=hardware_id, 
)
print(amount)
duration=3600*duration_hour

0


### Create task

This step uses above information to create task, and get `task_uuid`, which is useful in submit payment step.

In [4]:
job_source_uri = 'https://github.com/alphaflows/tetris-docker-image.git'

In [5]:
result = swan_orchestrator.create_task( 
    job_source_uri=job_source_uri,
    # hardware_id=hardware_id,
    wallet_address=wallet_address,
    # region='North Carolina-US',
    # duration=duration
    auto_pay=False,
    # image_name="hello world"
)
print(json.dumps(result, indent=2))
task_uuid = result['data']['task']['uuid']

0
https://plutotest.acl.swanipfs.com/ipfs/QmWDDBnYg2qdqLQ4o5WBvrjYL7ghTZeaKdKTqPyqcySBx3
{
  "data": {
    "task": {
      "comments": null,
      "created_at": 1719522313,
      "end_at": 1719525913,
      "id": 4407,
      "leading_job_id": null,
      "refund_amount": null,
      "refund_wallet": "0x00165Ca5Ab193c4D2709F94A5418165bA04394E6",
      "source": "v2",
      "start_at": 1719522313,
      "start_in": 300,
      "status": "initialized",
      "task_detail": {
        "amount": 0.0,
        "bidder_limit": 3,
        "created_at": 1719522313,
        "duration": 3600,
        "end_at": 1719525913,
        "hardware": "C1ae.small",
        "job_result_uri": null,
        "job_source_uri": "https://plutotest.acl.swanipfs.com/ipfs/QmWDDBnYg2qdqLQ4o5WBvrjYL7ghTZeaKdKTqPyqcySBx3",
        "price_per_hour": "0.0",
        "requirements": {
          "hardware": "None",
          "hardware_type": "CPU",
          "memory": "2",
          "region": "global",
          "vcpu": "2"
  

### Submit Payment

This step is using `task_uuid`, `hardware_id` and `duration` to submit payment via **ClientPayment** contract.

In [11]:
# tx_hash = contract.submit_payment(task_uuid, hardware_id, duration)
# print(tx_hash)

### Validate Payment to deploy task

This step will validate the payment and then make task eligible for assigning if validation successful

In [12]:
# if result_validation := swan_api.validate_payment(
#     tx_hash=tx_hash,
#     task_uuid=task_uuid
# ):
#     print(json.dumps(result_validation, indent=2))
# else:
#     print('validation failed')

### Make Payment to deploy task

This step will make the payment and then make task eligible for assigning if payment successful

In [6]:
duration=3600
if result_validation := swan_orchestrator.make_payment(task_uuid=task_uuid, 
                                              duration=duration, 
                                              primary_key=primary_key
                                              # hardware_id=hardware_id
                                              ):
    print(json.dumps(result_validation, indent=2))
else:
    print('validation failed')

hardware id is 0
{'tx_hash': '0x23ca76a7faa7be3362cd15e651c12d571f933c724d1397f369bc6e5501372d8d', 'task_uuid': '91dfd96c-8c33-4353-acfb-94c20d257bad'}
{
  "data": {
    "task": {
      "comments": null,
      "created_at": 1719522313,
      "end_at": 1719525913,
      "id": 4407,
      "leading_job_id": null,
      "refund_amount": null,
      "refund_wallet": "0x00165Ca5Ab193c4D2709F94A5418165bA04394E6",
      "source": "v2",
      "start_at": 1719522313,
      "start_in": 300,
      "status": "created",
      "task_detail": {
        "amount": 0.0,
        "bidder_limit": 3,
        "created_at": 1719522313,
        "duration": 3600,
        "end_at": 1719525913,
        "hardware": "C1ae.small",
        "job_result_uri": null,
        "job_source_uri": "https://plutotest.acl.swanipfs.com/ipfs/QmWDDBnYg2qdqLQ4o5WBvrjYL7ghTZeaKdKTqPyqcySBx3",
        "price_per_hour": "0.0",
        "requirements": {
          "hardware": "None",
          "hardware_type": "CPU",
          "memory": 

### follow up task status (optional)
The following step is optional, shows information when waiting for task being deployed.

In [8]:
# Check task info
info = swan_orchestrator.get_deployment_info(task_uuid=task_uuid)
print(json.dumps(info, indent=2))

{
  "data": {
    "computing_providers": [
      {
        "beneficiary": "0xFbc1d38a2127D81BFe3EA347bec7310a1cfa2373",
        "cp_account_address": "0x4D81F4277667862046802bDdcdB667704bEFbbAB",
        "created_at": 1718385670,
        "id": 66,
        "lat": 45.5075,
        "lon": -73.5887,
        "multi_address": [
          "/ip4/38.140.46.60/tcp/8086"
        ],
        "node_id": "0478a886cb6e191a0531fb6f8a7da4c0fbf0ec220af6d49e622f7e2da514f0ea205ed77918796c66ae642f0996dbf0ecc4bb7f1dd949705d1306abd1c14b60fb8a",
        "online": 1,
        "owner_address": "0xFbc1d38a2127D81BFe3EA347bec7310a1cfa2373",
        "region": "Quebec-CA",
        "task_types": "[1, 2, 3]",
        "updated_at": 1719522331,
        "version": "2.0",
        "worker_address": "0xFbc1d38a2127D81BFe3EA347bec7310a1cfa2373"
      },
      {
        "beneficiary": "0x1AE8665873565C9089753920A0866C49Bd35DCC9",
        "cp_account_address": "0xd63853b23Af4cc3b37377368dd33F09b8567aCea",
        "created_at": 

### Show result

`job_real_uri` is for show the result of application you deployed.  
You can put it into the web browser to view application.

In [9]:
r = swan_orchestrator.get_real_url(task_uuid)
print(r)

['https://j4hjf5ky4d.dev2.crosschain.computer', 'https://igmlg02rty.cp.filezoo.com.cn']


In [11]:
renew_task = swan_orchestrator.renew_task(task_uuid=task_uuid, 
                                 duration=duration * 2,
                                 # hardware_id=hardware_id
                                )
print(renew_task)

ERROR:root:SwanAPIRequestException: Invalid hardware_id, please provide a hardware_id or set with set_configTraceback (most recent call last):
  File "c:\Users\GeorgeMo\Documents\python-swan-sdk\examples\..\swan\api\swan_api.py", line 539, in renew_task
    raise SwanAPIException(f"Invalid hardware_id, please provide a hardware_id or set with set_config")
swan.common.exception.SwanAPIException: SwanAPIRequestException: Invalid hardware_id, please provide a hardware_id or set with set_config



None


In [12]:
r = swan_orchestrator.terminate_task(task_uuid)
print(r)

{'data': {'retryable': False, 'task_status': 'terminate_pending'}, 'message': 'Task termination is queued', 'status': 'success'}


In [21]:
claim_review = swan_orchestrator.claim_review(task_uuid)
print(claim_review)

{'data': {'error_code': 1022}, 'message': "task_uuid='a5bc7071-7f3f-46c8-9df6-732bc781addb' all payments (completed) not in cooling down status, cannot claim review", 'status': 'failed'}


In [10]:
import requests
import json

headers = {
    'Content-Type': 'application/json',
}

response = requests.get(r[0], headers=headers)

try:
    print(json.dumps(response.json(), indent=4))
except Exception as e:
    print(e)
    print(response)


Expecting value: line 1 column 1 (char 0)
<Response [200]>
